# Multi-language attacks

TextAttack's four-component framework makes it trivial to run attacks in other languages. In this tutorial, we:

- Create a model wrapper around Transformers [pipelines](https://huggingface.co/transformers/main_classes/pipelines.html) 
- Initialize a pre-trained [CamemBERT](https://camembert-model.fr/) model for sentiment classification
- Load the AlloCiné movie review sentiment classification dataset (from [`datasets`](https://github.com/huggingface/datasets/))
- Load the `french` recipe, which uses WordNet to find synonyms
- Run an adversarial attack on a French language model

Voilà!

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QData/TextAttack/blob/master/docs/2notebook/Example_4_CamemBERT.ipynb)

[![View Source on GitHub](https://img.shields.io/badge/github-view%20source-black.svg)](https://github.com/QData/TextAttack/blob/master/docs/2notebook/Example_4_CamemBERT.ipynb)

In [1]:
!pip install textattack
!pip install tensorflow_text

In [2]:

from textattack.attack_recipes import FrenchRecipe
from textattack.datasets import HuggingFaceDataset
from textattack.models.wrappers import ModelWrapper
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, pipeline
from textattack import Attacker

import numpy as np

# Quiet TensorFlow.
import os
if "TF_CPP_MIN_LOG_LEVEL" not in os.environ:
    os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"


class HuggingFaceSentimentAnalysisPipelineWrapper(ModelWrapper):
    """ Transformers sentiment analysis pipeline returns a list of responses
        like 
        
            [{'label': 'POSITIVE', 'score': 0.7817379832267761}]
            
        We need to convert that to a format TextAttack understands, like
        
            [[0.218262017, 0.7817379832267761]
    """
    def __init__(self, model):
        self.model = model#pipeline = pipeline
    def __call__(self, text_inputs):
        raw_outputs = self.model(text_inputs)
        outputs = []
        for output in raw_outputs:
            score = output['score']
            if output['label'] == 'POSITIVE':
                outputs.append([1-score, score])
            else:
                outputs.append([score, 1-score])
        return np.array(outputs)


In [3]:
# Create the model: a French sentiment analysis model.
# see https://github.com/TheophileBlard/french-sentiment-analysis-with-bert
model = TFAutoModelForSequenceClassification.from_pretrained("tblard/tf-allocine")
tokenizer = AutoTokenizer.from_pretrained("tblard/tf-allocine")
pipeline = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

model_wrapper = HuggingFaceSentimentAnalysisPipelineWrapper(pipeline)

# Create the recipe: PWWS uses a WordNet transformation.
recipe = FrenchRecipe.build(model_wrapper)

dataset = HuggingFaceDataset('allocine', split='test')

attacker = Attacker(recipe, dataset)
attacker.attack_dataset()


Attempting to acquire lock 140469075791504 on /root/.cache/huggingface/transformers/0ff95e1742945580afa83294a191021317b5fe6295b5cb71e25ff60cc8db778a.970c7da56e98cae263b795936c42ac83bddf59f600e011b9202f48057ba4d745.lock
Lock 140469075791504 acquired on /root/.cache/huggingface/transformers/0ff95e1742945580afa83294a191021317b5fe6295b5cb71e25ff60cc8db778a.970c7da56e98cae263b795936c42ac83bddf59f600e011b9202f48057ba4d745.lock


Attempting to release lock 140469075791504 on /root/.cache/huggingface/transformers/0ff95e1742945580afa83294a191021317b5fe6295b5cb71e25ff60cc8db778a.970c7da56e98cae263b795936c42ac83bddf59f600e011b9202f48057ba4d745.lock
Lock 140469075791504 released on /root/.cache/huggingface/transformers/0ff95e1742945580afa83294a191021317b5fe6295b5cb71e25ff60cc8db778a.970c7da56e98cae263b795936c42ac83bddf59f600e011b9202f48057ba4d745.lock


Attempting to acquire lock 140469055522896 on /root/.cache/huggingface/transformers/36f37db4cdf95317cb7382ad5e849c22a3fddfe8d35d834708fffca8c094bdfd.87ea704484107e69e19d82f8177d0f52b885d868034f1d621df6c655adac3bcb.h5.lock
Lock 140469055522896 acquired on /root/.cache/huggingface/transformers/36f37db4cdf95317cb7382ad5e849c22a3fddfe8d35d834708fffca8c094bdfd.87ea704484107e69e19d82f8177d0f52b885d868034f1d621df6c655adac3bcb.h5.lock


Attempting to release lock 140469055522896 on /root/.cache/huggingface/transformers/36f37db4cdf95317cb7382ad5e849c22a3fddfe8d35d834708fffca8c094bdfd.87ea704484107e69e19d82f8177d0f52b885d868034f1d621df6c655adac3bcb.h5.lock
Lock 140469055522896 released on /root/.cache/huggingface/transformers/36f37db4cdf95317cb7382ad5e849c22a3fddfe8d35d834708fffca8c094bdfd.87ea704484107e69e19d82f8177d0f52b885d868034f1d621df6c655adac3bcb.h5.lock


All model checkpoint layers were used when initializing TFCamembertForSequenceClassification.

All the layers of TFCamembertForSequenceClassification were initialized from the model checkpoint at tblard/tf-allocine.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFCamembertForSequenceClassification for predictions without further training.
Attempting to acquire lock 140469132392656 on /root/.cache/huggingface/transformers/515008bc12fa4e2c79385479978d339b14491b0f493869a937bb093bfbcfed0f.5cc6e825eb228a7a5cfd27cb4d7151e97a79fb962b31aaf1813aa102e746584b.lock
Lock 140469132392656 acquired on /root/.cache/huggingface/transformers/515008bc12fa4e2c79385479978d339b14491b0f493869a937bb093bfbcfed0f.5cc6e825eb228a7a5cfd27cb4d7151e97a79fb962b31aaf1813aa102e746584b.lock


...

Attempting to release lock 140469130949520 on /root/.cache/huggingface/transformers/cbf7ab8e5f82e0468952b19750f1396f8eb654365ea7f0c2cd5e1180d87222fd.cb3ec3a6c1200d181228d8825ae9767572abca54efa1bbb37fd83d721b2ef323.lock
Lock 140469130949520 released on /root/.cache/huggingface/transformers/cbf7ab8e5f82e0468952b19750f1396f8eb654365ea7f0c2cd5e1180d87222fd.cb3ec3a6c1200d181228d8825ae9767572abca54efa1bbb37fd83d721b2ef323.lock


textattack: Unknown if model of class <class 'transformers.pipelines.text_classification.TextClassificationPipeline'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


textattack: Loading datasets dataset allocine, split test.
  0%|          | 0/10 [00:00<?, ?it/s]

Dataset allocine_dataset downloaded and prepared to /root/.cache/huggingface/datasets/allocine_dataset/allocine/1.0.0/91f700d606838c22c5c370846746e60503219d0c1f16ed96bfd1fa19a73458eb. Subsequent calls will reuse this data.
Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapWordNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



KeyboardInterrupt: ignored